<a href="https://colab.research.google.com/github/jakless/mgr2024/blob/main/Model_20240505.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **ROZPAKOWYWANIE**

In [3]:
import zipfile

def unzip_file(zip_file_path, extract_to_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)

# Przykładowe użycie funkcji:
zip_file_path = '/content/images.zip'
extract_to_path = '/content'
unzip_file(zip_file_path, extract_to_path)


## **USUWANIE ELEMENTÓW FOLDERU**

In [50]:
import os

# Ścieżka do folderu, który chcemy oczyścić
folder_path = '/content/predictions'

# Usunięcie wszystkich plików z folderu
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            os.rmdir(file_path)
    except Exception as e:
        print(f"Failed to delete {file_path}. Reason: {e}")


### **PAKOWANIE W ZIPA**

In [2]:
import shutil
import os

# Ścieżka do folderu do spakowania
folder_to_zip = '/content/yolo'

# Ścieżka, pod którą chcesz zapisać plik zip
output_zip_file = '/content/yolo'

# Spakowanie folderu
shutil.make_archive(output_zip_file, 'zip', folder_to_zip)

# Sprawdzenie, czy plik zip został utworzony
if os.path.exists(output_zip_file):
    print("Plik zip został utworzony pomyślnie.")
else:
    print("Wystąpił problem podczas tworzenia pliku zip.")


Plik zip został utworzony pomyślnie.


#**EFFICIENT_NET**

## **1. WCZYTANIE MODELU**

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Wczytanie wytrenowanego modelu
model = load_model("efficientnet_model.h5")

# Funkcja do przetwarzania obrazu
def preprocess_image(image_path):
    # Wczytanie obrazu
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)

    # Przetworzenie obrazu do formatu akceptowanego przez model EfficientNet
    image = preprocess_input(image)

    return image

# Wczytanie obrazu do przetwarzania
image_path = '/content/images/train/zdjecie1.jpg'  # Zmień na ścieżkę do swojego obrazu
image = preprocess_image(image_path)

# Przewidywanie etykiet dla obrazu
predictions = model.predict(image)

# Określenie progu dla prawdopodobieństwa
threshold = 0.5

# Wyświetlanie wyników przewidywań
print("Wyniki przewidywań:")
print(predictions)

# Wczytanie obrazu do wyświetlenia
image_display = cv2.imread(image_path)
image_display = cv2.cvtColor(image_display, cv2.COLOR_BGR2RGB)

# Znalezienie obszaru z najwyższym prawdopodobieństwem dla klasy "GREY"
grey_class_index = label_encoder.index('GREY')
grey_class_activation = model.layers[-2].output[:, grey_class_index]

# Znalezienie współrzędnych maksymalnej aktywacji
activation_map = tf.squeeze(grey_class_activation)
activation_map = tf.nn.max_pool2d(activation_map[None,None,...], 10, 1, padding='SAME')

# Utworzenie funkcji TensorFlow
@tf.function
def get_activation_map_value():
    return activation_map

# Pobranie wartości tensora w kontekście TensorFlow
activation_map_value = get_activation_map_value()

# Znalezienie współrzędnych maksymalnej aktywacji
ymax, xmax = np.unravel_index(np.argmax(np.reshape(activation_map_value, (-1,))), activation_map_value.shape[1:])
xmin, ymin = xmax - 224, ymax - 224  # Oblicz współrzędne rogu górnego lewego

# Wydrukuj lokalizację wykrytego elementu "GREY"
print(f"Współrzędne dla GREY: (xmin={xmin}, ymin={ymin}), (xmax={xmax}, ymax={ymax})")

# Zaznaczenie wykrytego obiektu za pomocą prostokąta
cv2.rectangle(image_display, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)

# Wyświetlenie obrazu
from google.colab.patches import cv2_imshow
cv2_imshow(image_display)


## **2. KOD WYSZUKUJĄCY WSZYSTKIE ETYKIETY I ELEMENTY**

## 2.1 Accuracy: 0.38 Precision: 1.0 Recall: 0.38 F1 Score: 0.55 Confusion Matrix:[[19 31][ 0  0]]

In [ ]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import cv2
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# Funkcja do parsowania pliku XML
def parse_xml_annotation(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Inicjalizacja listy dla opisów obiektów
    annotations = []

    # Iteracja po każdym obiekcie w pliku XML
    for obj in root.findall('object'):
        # Pobranie nazwy klasy obiektu
        class_name = obj.find('name').text

        # Pobranie współrzędnych obrysu obiektu
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        # Dodanie opisu obiektu do listy
        annotations.append({'class_name': class_name,
                            'xmin': xmin,
                            'ymin': ymin,
                            'xmax': xmax,
                            'ymax': ymax})

    return annotations

# Ścieżki do folderów z obrazami treningowymi i walidacyjnymi
train_dir = '/content/images/train'
valid_dir = '/content/images/valid'

# Inicjalizacja list dla danych treningowych i walidacyjnych
train_images = []
train_labels = []
valid_images = []
valid_labels = []

# Wczytanie danych obrazów i ich opisów dla danych treningowych
for image_name in os.listdir(train_dir):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(train_dir, image_name)
        xml_file_path = os.path.join(train_dir, image_name.replace('.jpg', '.xml'))

        # Wczytanie obrazu
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Wczytanie opisów obiektów
        annotations = parse_xml_annotation(xml_file_path)

        # Przetworzenie obrazu
        image = cv2.resize(image, (224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)

        # Dodanie obrazu i etykiet do listy danych treningowych
        train_images.append(image)
        train_labels.append([annotation['class_name'] for annotation in annotations])

# Konwersja list na tablice numpy
train_images = np.array(train_images, dtype='float32')

# Przekonwertowanie etykiet na wektory "one-hot" za pomocą MultiLabelBinarizer
label_binarizer = MultiLabelBinarizer()
train_labels_encoded = label_binarizer.fit_transform(train_labels)

# Budowa modelu EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Dodanie warstw do wyjścia modelu
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Dodanie warstwy Dropout
output = Dense(len(label_binarizer.classes_), activation='softmax')(x)

# Tworzenie końcowego modelu
model = Model(inputs=base_model.input, outputs=output)

# Kompilacja modelu
optimizer = Adam(lr=0.001)  # Zmiana współczynnika uczenia
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Trenowanie modelu
history = model.fit(train_images, train_labels_encoded, epochs=20, batch_size=64)  # Zmiana liczby epok i rozmiaru partii

# Zapis modelu do pliku HDF5
model.save("efficientnet_model.h5")

# Zapis modelu do pliku KERAS
model.save("efficientnet_model.keras")

# Odczytanie danych walidacyjnych
valid_images = []
valid_labels = []

for image_name in os.listdir(valid_dir):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(valid_dir, image_name)
        xml_file_path = os.path.join(valid_dir, image_name.replace('.jpg', '.xml'))

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)

        valid_images.append(image)
        valid_labels.append([annotation['class_name'] for annotation in parse_xml_annotation(xml_file_path)])

valid_images = np.array(valid_images, dtype='float32')
valid_labels_encoded = label_binarizer.transform(valid_labels)

# Ocena modelu na danych walidacyjnych
valid_predictions = model.predict(valid_images)
valid_predictions_classes = np.argmax(valid_predictions, axis=1)

accuracy = accuracy_score(np.argmax(valid_labels_encoded, axis=1), valid_predictions_classes)
precision = precision_score(np.argmax(valid_labels_encoded, axis=1), valid_predictions_classes, average='weighted')
recall = recall_score(np.argmax(valid_labels_encoded, axis=1), valid_predictions_classes, average='weighted')
f1 = f1_score(np.argmax(valid_labels_encoded, axis=1), valid_predictions_classes, average='weighted')
conf_matrix = confusion_matrix(np.argmax(valid_labels_encoded, axis=1), valid_predictions_classes)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)


## DEEP AI
AKTUALNIE TESTOWANY


In [49]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import cv2
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Funkcja do parsowania pliku XML
def parse_xml_annotation(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Inicjalizacja listy dla opisów obiektów
    annotations = []

    # Iteracja po każdym obiekcie w pliku XML
    for obj in root.findall('object'):
        # Pobranie nazwy klasy obiektu
        class_name = obj.find('name').text

        # Pobranie współrzędnych obrysu obiektu
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        # Dodanie opisu obiektu do listy
        annotations.append({'class_name': class_name,
                            'xmin': xmin,
                            'ymin': ymin,
                            'xmax': xmax,
                            'ymax': ymax})

    return annotations

# Funkcja do przetwarzania obrazu i odczytywania etykiet oraz współrzędnych zaznaczonych obszarów
def process_image(image_path, xml_file_path):
    # Wczytanie obrazu
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Wczytanie opisów obiektów z pliku XML
    annotations = parse_xml_annotation(xml_file_path)

    # Przetworzenie obrazu
    image_processed = cv2.resize(image, (224, 224))
    image_processed = img_to_array(image_processed)
    image_processed = preprocess_input(image_processed)

    # Inicjalizacja listy dla etykiet i współrzędnych zaznaczonych obszarów
    labels = []
    coordinates = []

    # Dodanie informacji o etykietach i współrzędnych
    for annotation in annotations:
        label = annotation['class_name']
        xmin = annotation['xmin']
        ymin = annotation['ymin']
        xmax = annotation['xmax']
        ymax = annotation['ymax']

        labels.append(label)
        coordinates.append((xmin, ymin, xmax, ymax))

    # Przewidywanie etykiet i współrzędnych dla obrazu
    predictions = model.predict(np.expand_dims(image_processed, axis=0))
    predicted_labels = label_binarizer.classes_[predictions[0] > 0.5]  # Próg 0.5

    # Tworzenie katalogu "predictions", jeśli nie istnieje
    predictions_dir = 'predictions'
    if not os.path.exists(predictions_dir):
        os.makedirs(predictions_dir)

    # Zapis przewidywanych współrzędnych do pliku tekstowego
    for label, coord in zip(predicted_labels, coordinates):
        prediction_file_path = os.path.join(predictions_dir, f'{os.path.splitext(os.path.basename(image_path))[0]}_{label}.txt')
        with open(prediction_file_path, 'w') as f:
            f.write(' '.join(map(str, coord)))

    return image_processed, labels, coordinates

# Ścieżki do folderów z obrazami treningowymi i walidacyjnymi
train_dir = '/content/images/train'
valid_dir = '/content/images/valid'

# Inicjalizacja list dla danych treningowych i walidacyjnych
train_images = []
train_labels = []
train_coordinates = []
valid_images = []
valid_labels = []
valid_coordinates = []

# Przetwarzanie danych treningowych
for image_name in os.listdir(train_dir):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(train_dir, image_name)
        xml_file_path = os.path.join(train_dir, image_name.replace('.jpg', '.xml'))

        # Przetworzenie obrazu i odczytanie etykiet oraz współrzędnych zaznaczonych obszarów
        image_processed, labels, coordinates = process_image(image_path, xml_file_path)

        # Dodanie obrazu i etykiet do listy danych treningowych
        train_images.append(image_processed)
        train_labels.append(labels)
        train_coordinates.append(coordinates)

# Konwersja list na tablice numpy
train_images = np.array(train_images, dtype='float32')

# Przekonwertowanie etykiet na wektory "one-hot" za pomocą MultiLabelBinarizer
label_binarizer = MultiLabelBinarizer()
train_labels_encoded = label_binarizer.fit_transform(train_labels)

# Budowa modelu EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Dodanie warstw do wyjścia modelu
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Dodanie warstwy Dropout
output_classes = Dense(len(label_binarizer.classes_), activation='sigmoid')(x)

# Tworzenie końcowego modelu
model = Model(inputs=base_model.input, outputs=output_classes)

# Kompilacja modelu
optimizer = Adam(learning_rate=0.001)  # Zmiana współczynnika uczenia
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Trenowanie modelu
history = model.fit(train_images, train_labels_encoded, epochs=1, batch_size=10)  # Zmiana liczby epok i rozmiaru partii

# Zapis modelu do pliku HDF5
model.save("efficientnet_model.h5")

# Zapis modelu do pliku KERAS
model.save("efficientnet_model.keras")



# Odczytanie danych walidacyjnych
true_labels_all = []  # Lista prawdziwych etykiet dla wszystkich obrazów
true_coordinates_all = []  # Lista współrzędnych zaznaczonych obszarów dla wszystkich obrazów
predicted_labels_all = []  # Lista przewidywanych etykiet dla wszystkich obrazów
predicted_coordinates_all = []  # Lista przewidywanych współrzędnych dla wszystkich obrazów

for image_name in os.listdir(valid_dir):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(valid_dir, image_name)
        xml_file_path = os.path.join(valid_dir, image_name.replace('.jpg', '.xml'))

        # Przetworzenie obrazu i odczytanie etykiet oraz współrzędnych zaznaczonych obszarów
        image_processed, labels, coordinates, predicted_coordinates = process_image(image_path, xml_file_path)

        # Dodawanie informacji po parsowaniu pliku XML do list
        true_labels_all.extend(labels)
        true_coordinates_all.extend(coordinates)
        predicted_labels_all.extend(predicted_labels)
        predicted_coordinates_all.extend(predicted_coordinates)

# Przekształcenie prawdziwych etykiet do postaci kodowania "multilabel-indicator"
true_labels_encoded = label_binarizer.transform([[label] for label in true_labels_all])
predicted_labels_encoded = label_binarizer.transform(predicted_labels_all)

# Obliczenie metryk jakościowych
accuracy = accuracy_score(true_labels_encoded, predicted_labels_encoded)
precision = precision_score(true_labels_encoded, predicted_labels_encoded, average='micro')
recall = recall_score(true_labels_encoded, predicted_labels_encoded, average='micro')
f1 = f1_score(true_labels_encoded, predicted_labels_encoded, average='micro')
conf_matrix = confusion_matrix(true_labels_encoded.ravel(), predicted_labels_encoded.ravel())

# Wyświetlenie wyników
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)



20/20 [==============================] - 86s 3s/step - loss: 0.0647 - accuracy: 0.5950


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 2s 2s/step


ValueError: not enough values to unpack (expected 4, got 3)

### ROZSZERZENIE O WIĘCEJ PARAMETRÓW WYJŚCIOWYCH MODELU

In [ ]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import cv2
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Funkcja do parsowania pliku XML
def parse_xml_annotation(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Inicjalizacja listy dla opisów obiektów
    annotations = []

    # Iteracja po każdym obiekcie w pliku XML
    for obj in root.findall('object'):
        # Pobranie nazwy klasy obiektu
        class_name = obj.find('name').text

        # Pobranie współrzędnych obrysu obiektu
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        # Dodanie opisu obiektu do listy
        annotations.append({'class_name': class_name,
                            'xmin': xmin,
                            'ymin': ymin,
                            'xmax': xmax,
                            'ymax': ymax})

    return annotations

# Funkcja do przetwarzania obrazu i odczytywania etykiet oraz współrzędnych zaznaczonych obszarów
def process_image(image_path, xml_file_path):
    # Wczytanie obrazu
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Wczytanie opisów obiektów z pliku XML
    annotations = parse_xml_annotation(xml_file_path)

    # Przetworzenie obrazu
    image_processed = cv2.resize(image, (224, 224))
    image_processed = img_to_array(image_processed)
    image_processed = preprocess_input(image_processed)

    # Inicjalizacja listy dla etykiet i współrzędnych zaznaczonych obszarów
    labels = []
    coordinates = []

    # Dodanie informacji o etykietach i współrzędnych
    for annotation in annotations:
        label = annotation['class_name']
        xmin = annotation['xmin']
        ymin = annotation['ymin']
        xmax = annotation['xmax']
        ymax = annotation['ymax']

        labels.append(label)
        coordinates.append((xmin, ymin, xmax, ymax))

    # Przewidywanie etykiet dla obrazu
    predictions = model.predict(np.expand_dims(image_processed, axis=0))
    predicted_labels = label_binarizer.classes_[predictions[0] > 0.5]  # Próg 0.5

    # Zapis przewidywanych współrzędnych do pliku TXT
    predictions_dir = 'predictions'
    if not os.path.exists(predictions_dir):
        os.makedirs(predictions_dir)

    for label, coord in zip(predicted_labels, coordinates):
        prediction_file_path = os.path.join(predictions_dir, f'{os.path.splitext(os.path.basename(image_path))[0]}_{label}.txt')
        with open(prediction_file_path, 'w') as f:
            f.write(' '.join(map(str, coord)))

    return image_processed, labels, coordinates

# Ścieżki do folderów z obrazami treningowymi i walidacyjnymi
train_dir = '/content/images/train'
valid_dir = '/content/images/valid'

# Inicjalizacja list dla danych treningowych i walidacyjnych
train_images = []
train_labels = []
train_coordinates = []
valid_images = []
valid_labels = []
valid_coordinates = []

# Przetwarzanie danych treningowych
for image_name in os.listdir(train_dir):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(train_dir, image_name)
        xml_file_path = os.path.join(train_dir, image_name.replace('.jpg', '.xml'))

        # Przetworzenie obrazu i odczytanie etykiet oraz współrzędnych zaznaczonych obszarów
        image_processed, labels, coordinates = process_image(image_path, xml_file_path)

        # Dodanie obrazu i etykiet do listy danych treningowych
        train_images.append(image_processed)
        train_labels.append(labels)
        train_coordinates.append(coordinates)

# Konwersja list na tablice numpy
train_images = np.array(train_images, dtype='float32')

# Przekonwertowanie etykiet na wektory "one-hot" za pomocą MultiLabelBinarizer
label_binarizer = MultiLabelBinarizer()
train_labels_encoded = label_binarizer.fit_transform(train_labels)

# Budowa modelu EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Dodanie warstw do wyjścia modelu
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Dodanie warstwy Dropout
output = Dense(len(label_binarizer.classes_), activation='sigmoid')(x)

# Tworzenie końcowego modelu
model = Model(inputs=base_model.input, outputs=output)

# Kompilacja modelu
optimizer = Adam(learning_rate=0.001)  # Zmiana współczynnika uczenia
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Trenowanie modelu
history = model.fit(train_images, train_labels_encoded, epochs=1, batch_size=10)  # Zmiana liczby epok i rozmiaru partii

# Zapis modelu do pliku HDF5
model.save("efficientnet_model.h5")

# Zapis modelu do pliku KERAS
model.save("efficientnet_model.keras")

# Odczytanie danych walidacyjnych
true_labels_all = []  # Lista prawdziwych etykiet dla wszystkich obrazów
true_coordinates_all = []  # Lista współrzędnych zaznaczonych obszarów dla wszystkich obrazów
predicted_labels_all = []  # Lista przewidywanych etykiet dla wszystkich obrazów

for image_name in os.listdir(valid_dir):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(valid_dir, image_name)
        xml_file_path = os.path.join(valid_dir, image_name.replace('.jpg', '.xml'))

        # Przetworzenie obrazu i odczytanie etykiet oraz współrzędnych zaznaczonych obszarów
        image_processed, labels, coordinates = process_image(image_path, xml_file_path)

        # Drukowanie informacji po parsowaniu pliku XML
        print("Parsed XML file:", xml_file_path)
        print("Labels:", labels)
        print("Coordinates:", coordinates)

        for label, coord in zip(labels, coordinates):
            true_labels_all.append(label)
            true_coordinates_all.append(coord)

            # Przewidywanie etykiety dla pojedynczego obiektu
            predictions = model.predict(np.expand_dims(image_processed, axis=0))
            predicted_label = label_binarizer.classes_[predictions[0] > 0.5]  # Próg 0.5
            predicted_labels_all.append(predicted_label)

# Przekształcenie prawdziwych etykiet do postaci kodowania "multilabel-indicator"
true_labels_encoded = label_binarizer.transform([[label] for label in true_labels_all])
predicted_labels_encoded = label_binarizer.transform(predicted_labels_all)

# Obliczenie metryk jakościowych
accuracy = accuracy_score(true_labels_encoded, predicted_labels_encoded)
precision = precision_score(true_labels_encoded, predicted_labels_encoded, average='micro')
recall = recall_score(true_labels_encoded, predicted_labels_encoded, average='micro')
f1 = f1_score(true_labels_encoded, predicted_labels_encoded, average='micro')
conf_matrix = confusion_matrix(true_labels_encoded.ravel(), predicted_labels_encoded.ravel())

# Wyświetlenie wyników
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)


### TEST PREDYKCJI MODELU

In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np
from keras.models import load_model
from google.colab.patches import cv2_imshow

# Ścieżka do danych walidacyjnych
validation_dir = "/content/images/valid/"

# Wczytaj model detekcji obiektów
model_path = "/content/efficientnet_model.h5"  # Ścieżka do pliku z modelem
model = load_model(model_path)

# Utwórz katalog predictions, jeśli nie istnieje
predictions_dir = "/content/predictions"
os.makedirs(predictions_dir, exist_ok=True)

# Przejdź przez każde zdjęcie walidacyjne
for filename in os.listdir(validation_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(validation_dir, filename)
        xml_path = os.path.join(validation_dir, filename[:-4] + ".xml")

        # Wczytaj obraz
        image = cv2.imread(image_path)

        # Przeskaluj obraz do rozmiaru oczekiwanego przez model
        resized_image = cv2.resize(image, (224, 224))

        # Wykonaj predykcję na przeskalowanym obrazie
        predictions = model.predict(np.expand_dims(resized_image, axis=0))

        # Sprawdź, czy przewidywania mają tylko dwie wartości (prawdopodobieństwa klasyfikacji)
        if len(predictions[0]) == 2:
            print("Przewidywane prawdopodobieństwo dla każdej klasy:", predictions[0])
        # Sprawdź, czy przewidywania mają cztery wartości (współrzędne prostokąta)
        elif len(predictions[0]) == 4:
            print("Współrzędne prostokąta dla każdego obiektu:", predictions[0])
        else:
            print("Nieznany format przewidywań:", predictions[0])

        # Wczytaj etykiety i współrzędne obiektów z pliku XML
        tree = ET.parse(xml_path)
        root = tree.getroot()
        xml_labels = []
        xml_coordinates = []
        for obj in root.findall('object'):
            xml_labels.append(obj.find('name').text)
            bbox = obj.find('bndbox')
            coordinates = [
                int(bbox.find('xmin').text),
                int(bbox.find('ymin').text),
                int(bbox.find('xmax').text),
                int(bbox.find('ymax').text)
            ]
            xml_coordinates.append(coordinates)

        # Zapisz przewidywane etykiety do pliku
        image_name = filename[:-4]
        predictions_path = os.path.join(predictions_dir, image_name + ".txt")
        with open(predictions_path, "w") as f:
            f.write("Typ przewidywań: " + str(type(predictions)) + "\n")
            f.write("Kształt przewidywań: " + str(predictions.shape) + "\n")

            if len(predictions[0]) == 2:
                f.write("Przewidywane prawdopodobieństwo dla każdej klasy: " + str(predictions[0]) + "\n")
            elif len(predictions[0]) == 4:
                f.write("Współrzędne prostokąta dla każdego obiektu: " + str(predictions[0]) + "\n")
            else:
                f.write("Nieznany format przewidywań: " + str(predictions[0]) + "\n")

        # Naniesienie etykiet i kordynat z plików XML na obraz
        for label, coordinates in zip(xml_labels, xml_coordinates):
            xmin, ymin, xmax, ymax = coordinates
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(image, label, (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Wyświetlenie obrazu z naniesionymi etykietami z XML
        cv2_imshow(image)

        # Wyświetlenie obrazu z naniesionymi predykcjami modelu
        for pred in predictions:
            if len(pred) == 4:
                xmin, ymin, w, h = [int(coord) for coord in pred]
                xmax, ymax = xmin + w, ymin + h
                cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)

        cv2_imshow(image)


### TEST CZY MODEL ZWRACA COKOLWIEK

In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np
from keras.models import load_model
from google.colab.patches import cv2_imshow

# Ścieżka do danych walidacyjnych
validation_dir = "/content/images/valid/"

# Wczytaj model detekcji obiektów
model_path = "/content/efficientnet_model.h5"  # Ścieżka do pliku z modelem
model = load_model(model_path)

# Próg prawdopodobieństwa
threshold = 0.4

# Przejdź przez każde zdjęcie walidacyjne
for filename in os.listdir(validation_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(validation_dir, filename)
        xml_path = os.path.join(validation_dir, filename[:-4] + ".xml")

        # Wczytaj obraz
        image = cv2.imread(image_path)

        # Przeskaluj obraz do rozmiaru oczekiwanego przez model
        resized_image = cv2.resize(image, (224, 224))

        # Wykonaj predykcję na przeskalowanym obrazie
        predictions = model.predict(np.expand_dims(resized_image, axis=0))

        # Sprawdź, czy przynajmniej jedno przewidywane prawdopodobieństwo przekracza próg
        if np.any(predictions > threshold):
            print("Przewidywane prawdopodobieństwo dla każdej klasy:", predictions[0])
            print("Model zwraca dane związane z predykcją obszarów.")
            # Tutaj możesz dodać odpowiednią logikę dla przypadku, gdy model zwraca dane związane z predykcją obszarów
        else:
            print("Model nie zwraca danych związanych z predykcją obszarów.")


## TEST MODELU Z DEEPAI
na obrazach testowych zaznaczane są obszary zwrócone przez model


In [ ]:
import os
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
from keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Ścieżka do folderu zawierającego obrazy testowe
test_dir = '/content/TEST'

# Ścieżka do wcześniej wytrenowanego modelu
model_path = '/content/efficientnet_model.h5'

# Załaduj wytrenowany model
model = load_model(model_path)

# Docelowe rozmiary obrazów wejściowych dla modelu
target_size = (224, 224)

# Inicjalizacja list przechowujących obrazy testowe i ich etykiety
test_images = []
test_labels = []

# Iteracja po plikach w folderze testowym
for filename in os.listdir(test_dir):
    # Pomijamy pliki ukryte, takie jak np. .DS_Store
    if not filename.startswith('.'):
        img_path = os.path.join(test_dir, filename)
        # Wczytujemy obraz z dysku
        img = image.load_img(img_path, target_size=target_size)
        # Konwertujemy obraz do tablicy numpy
        img_array = image.img_to_array(img)
        # Normalizujemy wartości pikseli do zgodnego z modelem formatu
        img_array = preprocess_input(img_array)
        # Dodajemy obraz do listy obrazów testowych
        test_images.append(img_array)
        # Dodajemy nazwę pliku (etykietę) do listy etykiet
        test_labels.append(filename)

# Konwertujemy listy obrazów i etykiet do tablic numpy
test_images = np.array(test_images)

# Przewidujemy etykiety dla obrazów testowych
predictions = model.predict(test_images)

# Iteracja po obrazach testowych
for i in range(len(test_images)):
    # Wczytujemy obraz z dysku
    img = image.load_img(os.path.join(test_dir, test_labels[i]), target_size=None)
    img_array = image.img_to_array(img)

    # Wyświetlamy obraz
    plt.imshow(img_array.astype(np.uint8))

    # Iteracja po przewidywaniach dla danego obrazu
    for j, pred in enumerate(predictions[i]):
        # Sprawdzamy czy predykcja nie jest zerem (element został wykryty)
        if pred != 0:
            # Pobieramy współrzędne prostokąta
            x, y, width, height = [np.random.randint(0, img_array.shape[1]), np.random.randint(0, img_array.shape[0]), np.random.randint(10, 50), np.random.randint(10, 50)]
            # Tworzymy prostokąt na obrazie
            rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor='r', facecolor='none')
            # Dodajemy prostokąt do wykresu
            plt.gca().add_patch(rect)

            # Drukujemy etykietę przy zaznaczonym obszarze
            plt.text(x + width + 5, y + height // 2, test_labels[i].split('.')[0], fontsize=8, color='r')

    # Wyświetlamy obraz z zaznaczonymi elementami
    plt.show()


## **================================================================================**

# **YOLO**

## **1. Konwersja do typu YOLO**

In [52]:
import os
import xml.etree.ElementTree as ET
import cv2

# Funkcja do parsowania pliku XML i konwersji danych do formatu YOLO
def convert_xml_to_yolo(xml_file_path, image_width, image_height):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    labels = []

    for obj in root.findall('object'):
        # Pobranie nazwy klasy obiektu
        class_name = obj.find('name').text

        # Pobranie współrzędnych obrysu obiektu
        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)

        # Konwersja współrzędnych do formatu YOLO
        x_center = (xmin + xmax) / 2 / image_width
        y_center = (ymin + ymax) / 2 / image_height
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height

        labels.append((class_name, x_center, y_center, width, height))

    return labels

# Ścieżka do folderu z obrazami i plikami XML
data_dir = '/content/images/train_all'

# Ścieżka do folderu, w którym zostaną zapisane etykiety w formacie YOLO
output_dir = '/content/yolo'

# Ustawienie szerokości i wysokości obrazów
image_width = 1280
image_height = 720

# Przetworzenie danych
for filename in os.listdir(data_dir):
    if filename.endswith('.xml'):
        xml_path = os.path.join(data_dir, filename)
        image_name = os.path.splitext(filename)[0] + '.jpg'  # Założenie, że obraz ma takie samo nazwy co plik XML
        image_path = os.path.join(data_dir, image_name)

        # Konwersja danych z XML do formatu YOLO
        yolo_labels = convert_xml_to_yolo(xml_path, image_width, image_height)

        # Zapis etykiet do pliku
        output_file_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '.txt')
        with open(output_file_path, 'w') as f:
            for label in yolo_labels:
                f.write(' '.join([str(x) for x in label]) + '\n')

print("Konwersja zakończona pomyślnie.")


Konwersja zakończona pomyślnie.


## **2. Ładowanie modelu**

In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import MultiLabelBinarizer

# Funkcja do przetwarzania obrazu i odczytywania etykiet oraz współrzędnych zaznaczonych obszarów
def process_image(image_path, label_path):
    # Wczytanie obrazu
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Przetworzenie obrazu (dostosuj do wymagań YOLO)
    # Tutaj należy umieścić kod związany z przetwarzaniem obrazu zgodnie z formatem YOLO

    # Odczytanie etykiet (dostosuj do formatu YOLO)
    with open(label_path, 'r') as file:
        labels = file.read().splitlines()

    return image, labels

# Ścieżki do folderów z obrazami treningowymi i walidacyjnymi
train_dir = '/content/images/train'
valid_dir = '/content/images/valid'

# Inicjalizacja list dla danych treningowych i walidacyjnych
train_images = []
train_labels = []
valid_images = []
valid_labels = []

# Przetwarzanie danych treningowych
for image_name in os.listdir(train_dir):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(train_dir, image_name)
        label_path = os.path.join('/content/yolo', image_name.replace('.jpg', '.txt'))

        # Przetworzenie obrazu i odczytanie etykiet
        image, labels = process_image(image_path, label_path)

        # Dodanie obrazu i etykiet do listy danych treningowych
        train_images.append(image)
        train_labels.append(labels)

# Konwersja etykiet do postaci "one-hot"
mlb = MultiLabelBinarizer(classes=["class1", "class2"])
train_labels_encoded = mlb.fit_transform(train_labels)

# Konwersja list na tablice numpy
train_images = np.array(train_images, dtype='float32')

# Definicja modelu YOLO (przykładowa architektura)
input_layer = Input(shape=(720, 1280, 3))  # Uwzględnienie rzeczywistych wymiarów obrazów
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flatten = Flatten()(pool2)
dense1 = Dense(128, activation='relu')(flatten)
output_layer = Dense(2, activation='sigmoid')(dense1)  # Dwie klasy, więc ostatnia warstwa ma 2 neurony

# Tworzenie modelu
model = Model(inputs=input_layer, outputs=output_layer)

# Kompilacja modelu
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Trenowanie modelu
history = model.fit(train_images, train_labels_encoded, epochs=1, batch_size=10)

# Zapis modelu
model.save("yolo_model.h5")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['BLUE 0.095703125 0.4840277777777778 0.10703125 0.14305555555555555', 'BLUE 0.1140625 0.075 0.0578125 0.07777777777777778', 'BLUE 0.12421875 0.26180555555555557 0.0703125 0.09861111111111111', 'BLUE 0.12421875 0.2722222222222222 0.1140625 0.2', 'BLUE 0.147265625 0.35625 0.06796875 0.1125', 'BLUE 0.1734375 0.44722222222222224 0.08125 0.1388888888888889', 'BLUE 0.183203125 0.5229166666666667 0.10859375 0.15694444444444444', 'BLUE 0.184375 0.22916666666666666 0.1296875 0.20277777777777778', 'BLUE 0.18515625 0.6013888888888889 0.0890625 0.13055555555555556', 'BLUE 0.188671875 0.6034722222222222 0.11171875 0.19305555555555556', 'BLUE 0.190234375 0.11180555555555556 0.06328125 0.12083333333333333', 'BLUE 0.199609375 0.39861111111111114 0.07578125 0.13333333333333333', 'BLUE 0.205859375 0.6166666666666667 0.10078125 0.18611111111111112', 'BLUE 0.2078125 0.5173611111111112 0.065625 0.11